In [1]:
import ski_hill_database_utils
import utils
DB_PATH = "ski_hill.db"

Populate database with synthetic data

In [2]:
ski_hill_database_utils.create_ski_hill_db(DB_PATH, n_huts=20)

SQLite database 'ski_hill.db' created with a 'mountain_huts' table and 'trails' table.


In [3]:
schema_text = ski_hill_database_utils.get_schema(DB_PATH)
utils.print_html(schema_text, f"Table Schemas in {DB_PATH}")

Get a sense of the data by showing the first 5 entries in each table

In [4]:
query_all_mountain_huts = "select * from mountain_huts limit 5;"
all_mountain_huts = ski_hill_database_utils.execute_sql(
    query=query_all_mountain_huts,
    db_path=DB_PATH)
utils.print_html(all_mountain_huts, query_all_mountain_huts)
query_all_trails = "select * from trails limit 5;"
all_trails = ski_hill_database_utils.execute_sql(
    query=query_all_trails,
    db_path=DB_PATH)
utils.print_html(all_trails, query_all_trails)

id,name,altitude
1,Septuagint Goodall,13518
2,Goodall Base,9642
3,Llama Jackolantern II,10373
4,Zephyr Tibult,12502
5,Deepti Abby,10699


hut1,hut2
16,1
3,2
17,3
10,4
7,5


Write SQL query to propose new ski routes, based on the existing `mountain_huts` and `trails`.

In [6]:
query = """
    with bd_trails as (
        select hut1, hut2 from trails
        union
        select hut2 as hut1, hut1 as hut2 from trails
    )
    select starting.name as startpt, middle.name as middlept, ending.name as endpt from bd_trails first_segment
    join bd_trails second_segment on first_segment.hut2 = second_segment.hut1
    join mountain_huts starting on starting.id = first_segment.hut1
    join mountain_huts middle on middle.id = first_segment.hut2
    join mountain_huts ending on ending.id = second_segment.hut2
    where starting.altitude > middle.altitude and middle.altitude > ending.altitude
    """
results = ski_hill_database_utils.execute_sql(
    query=query,
    db_path=DB_PATH)
utils.print_html(results, "Proposed Routes")

startpt,middlept,endpt
Deepti Abby,Eileen Eileen,Chalet Frederic
Deepti Pecunia,Eileen Eileen,Chalet Frederic
Septuagint Goodall,Koala Septuagint,MarinoJ Vail
Septuagint Ishi,Koala Septuagint,MarinoJ Vail
Chalet Frederic 2,Koala Septuagint,MarinoJ Vail
Tibult Henrik,Chalet Frederic 2,Goodall Base
Tibult Henrik,Chalet Frederic 2,Koala Septuagint
Deepti Pecunia,Xerxes Jackolantern,Base Frederic II


Add summary information for the altitude change

In [ ]:
query = """
    with bd_trails as (
        select hut1, hut2 from trails
        union
        select hut2 as hut1, hut1 as hut2 from trails
    )
    select starting.name as startpt, middle.name as middlept, ending.name as endpt
    from bd_trails first_segment
    join bd_trails second_segment on first_segment.hut2 = second_segment.hut1
    join mountain_huts starting on starting.id = first_segment.hut1
    join mountain_huts middle on middle.id = first_segment.hut2
    join mountain_huts ending on ending.id = second_segment.hut2
    where starting.altitude > middle.altitude and middle.altitude > ending.altitude
    """
results = ski_hill_database_utils.execute_sql(
    query=query,
    db_path=DB_PATH)
utils.print_html(results, "Proposed Routes and Altitude Change")

Notice that not all `mountain_huts` are in use. Can you find which huts are left off the list? We may want to close those down, or find some other use for them.

What if some of the `mountain_huts` are in a state of disrepair?

In [ ]:
# update the database with a new column for hut quality,
#   based on "survey" done by experts (just randomly generate it, lol)
update_query = """
    UPDATE mountain_huts
    """

What about the quality of the existing routes? Are they a bunny slope, cat track, green, blue, black, double black, backcountry?

What about current conditions? Are the routes powder (fresh, untracked snow), moguls(bumps on a ski run), gnarly (extreme conditions)

Let's incorporate a table of weather data